### Maestría en Ciencia de Datos, ITAM

  **Curso de Optimización 2 2021-1,"Optimización Avanzada"**  
  **Prof. Erick Palacios Moreno**

> * Equipo 5:  
MIGUEL LOPEZ  
CARLOS LOPEZ  
JOSÉ ZARATE

**EXPERIMENTACIÓN. Uso de herramientas de kale, kubeflow, minikube para detectar con qué valores no funcionan sus programas.**

El paquete que desarrollamos se compone de dos métodos de optimización convexa, uno metaheurístico _Hill Climbing_ y uno por descenso en gradiente _Conjugate Gradient_. En este reporte, presentamos _Hill Climbing_ aplicado para resolver el problema del viajero (TSP) por medio del análisis de la ruta más corta por distancia euclideana. En una segunda etapa se modificará para estimación de distancias de coordenadas terrestres "_Great Circle Distance_".

____
## RESOLVER EL PROBLEMA DE TRAVELING SALESMAN PROBLEM (A.K.A. TSP) PARA DISTINTAS CIUDADES UTILIZANDO EL MÉTODO DE HILL CLIMBING

 * Distancia Euclideana.

> Dataset: [National Traveling Salesman Problems, CANADA](https://www.math.uwaterloo.ca/tsp/world/countries.html)

-------

### Hill Climbing 

El algoritmo de hill-climbing realiza seguimiento de un estado actual y en cada iteración se mueve al estado vecino con el valor más alto o menor, según sea el caso, es decir, se dirige en la dirección que proporciona el ascenso o descenso de mayor inclinación. Termina cuando alcanza un máximo o mínimo local. Hill climbing no mira hacia el futuro más allá de los vecinos inmediatos del estado actual. Esto se asemeja a tratar de encontrar la cima del Monte Everest en una espesa niebla mientras sufre de amnesia. Toma en cuenta que una forma de utilizar hill-climbing  consiste en utilizar el negativo de una función de coste heurística como función objetivo; que ascenderá localmente al estado con menor distancia heurística a la meta.

Los problemas que tiene Hill Climbing son:

MÁXIMA LOCAL: Un máximo local es un pico que es más alto que los estados vecinos pero menor que el máximo global. Cuando este algoritmo alcanzan las proximidades de un máximo local no podrá escapar de este, amenos que estemos tratando con un problema convexo.

CORDILLERAS: Las cordilleras son un reto para los algoritmos de hill climbing que optimizan en espacios continuos. Debido a que los algoritmos de hill climbing ajustan solo un elemento en el vector a la vez, a cada paso se va a mover en una dirección alineada con el eje. Si la función objetivo determina una cordillera estrecha que asciende en una dirección no alineada con el eje, entonces el algoritmo de hill climbing solo puede ascender la cordillera en zigzag. Si los lados de la cordillera son muy pronunciados, entonces el algoritmo puede verse forzado a realizar pasos muy pequeños mientras zigzaguea hacia una mejor posición.

MESETA: Se encuentra una meseta cuando el espacio de búsqueda es plano o lo suficientemente plano como para que el valor devuelto por la función objetivo sea indistinguible del valor devuelto para las regiones cercanas debido a la precisión utilizada por la máquina para representar la misma.

-------
**MODIFICACIONES DERIVADAS DE LA ETAPA DE EXPERIMENTACIÓN:**

Se desarrollaron diversos experimentos para analizar la eficiencia del algoritmo, la tolerancia y el punto inicial. Observamos que una de las mayores dificultades a resolver era que nuestro algoritmo era ineficiente para un númer mayor a 9 ciudades para cualquier punto inicial y tolerancia asignada (norma-distncia entre dos rutas para elegir la de mayor eficiencia). Se realizó una recodificación del código a partir de consultar otros algoritmos desarrollados por la comunidad (ver README de nuestro repo) bajo los cuales logramos tiempos sumamente eficientes incluso con más de 100 ciudades.

En una segunda etapa buscaremos optimizar el código en la siguiente práctica por medio de un análisis de perfilamiento y la consecuente reprogramación; finalmente se implementará el cómputo en paralelo. 

----

In [1]:
import Opt_HC_CG.hill as opt
import pandas as pd
import numpy as np

**Transformaciones**

Durante el proceso de exeperimentación se detectó la ineficiencia del algoritmo y se realizó una reprogramación para lograr tener tiempos aceptables. Se logró reducir de forma considerable, por lo que para mostrar nuestros resultados realizaremos una ejecución para 100 ciudades:

In [2]:
raw_data = pd.read_csv("../datasets/ca4663.tsp", sep = " ", names = ['index','x','y'])

In [ ]:
raw_data1 = raw_data.drop(['index'], axis = 1)
raw_data2 = raw_data1.dropna()
tsp_cities = raw_data2.iloc[0:50,].to_numpy()

Observamos la estructura de los datos:

In [3]:
print(tsp_cities)

[[41800.     82650.    ]
 [41966.6667 82533.3333]
 [41983.3333 82933.3333]
 [42033.3333 82750.    ]
 [42033.3333 82916.6667]
 [42050.     82583.3333]
 [42100.     82450.    ]
 [42100.     83100.    ]
 [42133.3333 82750.    ]
 [42150.     82966.6667]
 [42166.6667 82816.6667]
 [42233.3333 82550.    ]
 [42250.     81916.6667]
 [42250.     82433.3333]
 [42283.3333 83066.6667]
 [42300.     82716.6667]
 [42300.     83066.6667]
 [42316.6667 82850.    ]
 [42316.6667 82900.    ]
 [42316.6667 82983.3333]
 [42316.6667 83000.    ]
 [42333.3333 82000.    ]
 [42333.3333 82950.    ]
 [42333.3333 83033.3333]
 [42400.     82183.3333]
 [42433.3333 81883.3333]
 [42550.     81966.6667]
 [42566.6667 81683.3333]
 [42583.3333 82166.6667]
 [42583.3333 82366.6667]
 [42600.     81600.    ]
 [42633.3333 80450.    ]
 [42650.     80816.6667]
 [42666.6667 81216.6667]
 [42700.     81083.3333]
 [42716.6667 81300.    ]
 [42716.6667 82483.3333]
 [42750.     81700.    ]
 [42783.3333 80200.    ]
 [42783.3333 80983.3333]


Los nodos anteriores constituyen las 100 primeras ciudades de nuestro dataset, de las cuales nuestro algoritmo determinará la distancia más corta:

In [4]:
best_distance, best_path, exec_time = opt.best_solution(tsp_cities, 1 , 1e-11)

**Resultados:**

   * Con base en Hill Climbing, la distancia más corta que ha encontrado el algoritmo es:

In [5]:
best_distance

30832.880624375633

   * La ruta óptima esta daterminada por los siguientes nodos-ciudad:

In [6]:
np.array(best_path)

array([37, 66, 74, 62, 65, 77, 90, 61, 48, 39, 12,  1,  0,  5,  6, 21, 25,
       32, 31, 47, 49, 51, 68, 69, 60, 52, 58, 54, 67, 89, 99, 95, 93, 96,
       97, 98, 82, 55, 56, 83, 72, 57, 50, 28, 11,  8, 10,  9, 20, 19, 22,
       18, 17, 15, 42, 76, 71, 92, 94, 87, 84, 63, 59, 46, 43, 36, 29, 24,
       13,  3,  4,  2,  7, 14, 16, 23, 26, 35, 40, 44, 38, 41, 79, 91, 81,
       78, 86, 70, 64, 80, 85, 88, 75, 73, 53, 45, 34, 33, 30, 27])

  * Finalmente, el tiempo de ejecución que le tomó al algoritmo para determinar la ruta más corta en 100 ciudades fue de (segundos):

In [7]:
print(exec_time)

64.63763737678528
